<a href="https://colab.research.google.com/github/rl-cyber/User-Engaged-Network-Diagnosis/blob/main/Reproduce_cellularlint_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/CellularLint/cellularlint-codes.git


Cloning into 'cellularlint-codes'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 127 (delta 66), reused 65 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (127/127), 2.17 MiB | 19.65 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [2]:
%cd cellularlint-codes

/content/cellularlint-codes


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Pretrained_Models.zip ./
!unzip ./Pretrained_Models.zip -d Pretrained_Models/

Archive:  ./Pretrained_Models.zip
 extracting: Pretrained_Models/roberta_weights.zip  
 extracting: Pretrained_Models/bert_weights.zip  
 extracting: Pretrained_Models/xlnet_weights.zip  


In [7]:
!chmod 700 unpack.sh
!./unpack.sh

Unzipped bert_weights.zip into bert_weights
Unzipped roberta_weights.zip into roberta_weights
Unzipped xlnet_weights.zip into xlnet_weights


In [8]:
!python3 tokenizer_and_sim_matrix.py 4G
!python3 tokenizer_and_sim_matrix.py 5G

tokenizer_config.json: 100% 1.19k/1.19k [00:00<00:00, 9.49MB/s]
config.json: 100% 594/594 [00:00<00:00, 4.61MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 1.44MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 2.82MB/s]
special_tokens_map.json: 100% 239/239 [00:00<00:00, 1.91MB/s]
2025-02-14 00:34:31.358217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739493271.641611   11826 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739493271.717437   11826 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 00:34:32.393621: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in pe

In [9]:
from transformers import BertModel, BertTokenizerFast

MODEL_SAVE_PATH = "/content/cellularlint-codes/saved_models/bert"

# Download and save BERT model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
model.save_pretrained(MODEL_SAVE_PATH)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print("BERT model and tokenizer saved at:", MODEL_SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BERT model and tokenizer saved at: /content/cellularlint-codes/saved_models/bert


In [10]:
import pandas as pd
import os
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, BertConfig, BertTokenizerFast
from tokenizers import BertWordPieceTokenizer
from torch import nn
np.random.seed(321)

BertModel.from_pretrained("/content/cellularlint-codes/Pretrained Models/bert_weights/")

ROOT_PATH = "/content/cellularlint-codes"
DATA_PATH = os.path.join(ROOT_PATH, "Data/SNLI")
PRETRAINED_PATH = os.path.join(ROOT_PATH, "Pretrained Models/bert_weights")
SAVE_MODEL_AT = os.path.join(ROOT_PATH, "saved_models/bert")
PRETRAINED_TOKENIZER = SAVE_MODEL_AT  # Tokenizer is usually saved in the same model folder
MODEL_PATH = None


df_train = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_train.csv"))
df_dev = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_dev.csv"))
df_test = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_test.csv"))


df_train = df_train.iloc[:500]
df_dev = df_train.copy().iloc[:100]


tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_TOKENIZER))
tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_PATH))
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


labels = {'contradiction':1,
          'entailment':0,
          'neutral':2,
          }
NUM_LABELS = len(labels)

def bert_encoding(sequence):
    return tokenizer.encode(sequence, add_special_tokens = False)
def str_to_int_list(data):
    return list(map(int, data))

df_train['token_type'] = df_train['token_type'].str.split()
df_train['token_type'] = df_train['token_type'].apply(str_to_int_list)

df_dev['token_type'] = df_dev['token_type'].str.split()
df_dev['token_type'] = df_dev['token_type'].apply(str_to_int_list)

df_test['token_type'] = df_test['token_type'].str.split()
df_test['token_type'] = df_test['token_type'].apply(str_to_int_list)

df_train['attention_mask'] = df_train['attention_mask'].str.split()
df_train['attention_mask'] = df_train['attention_mask'].apply(str_to_int_list)

df_dev['attention_mask'] = df_dev['attention_mask'].str.split()
df_dev['attention_mask'] = df_dev['attention_mask'].apply(str_to_int_list)

df_test['attention_mask'] = df_test['attention_mask'].str.split()
df_test['attention_mask'] = df_test['attention_mask'].apply(str_to_int_list)


df_train['input_ids'] = df_train['sequence'].apply(bert_encoding)
df_dev['input_ids'] = df_dev['sequence'].apply(bert_encoding)
df_test['input_ids'] = df_test['sequence'].apply(bert_encoding)


max_length = 512
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = []
        self.labels = [labels[label] for label in df['gold_label']]
        for _, row in df.iterrows():
            token_type_ids = row['token_type']
            token_type_ids += [0] * (max_length - len(token_type_ids))
            attention_mask = row['attention_mask']
            attention_mask += [0] * (max_length - len(attention_mask))
            input_ids = tokenizer.encode(
                row['sequence'],
                add_special_tokens=False,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            datadict = {
                'input_ids': input_ids.squeeze(0),
                'token_type_ids': torch.tensor(token_type_ids),
                'attention_mask': torch.tensor(attention_mask)
            }
            self.texts.append(datadict)

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_text = self.texts[idx]
        batch_y = self.labels[idx]
        return batch_text, batch_y


class BertClassifier(nn.Module):

    def __init__(self, load_path = None, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(os.path.join(ROOT_PATH, load_path))
        self.bert = BertModel.from_pretrained(load_path)

        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, NUM_LABELS)

        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_id, mask, token_type_id):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask = mask, token_type_ids = token_type_id,return_dict = False)

        dropout_output = self.dropout(pooled_output)

        linear_output = self.linear(dropout_output)

        final_layer = self.softmax(linear_output)

        return final_layer

    def save(self, save_dir, tokenizer, model_name = "model_bert.pt"):

        os.makedirs(save_dir, exist_ok=True)
        # Save model weights
        model_path = os.path.join(save_dir, model_name)
        torch.save(self.state_dict(), model_path)
        self.bert.save_pretrained(save_dir)

        # Save tokenizer
        tokenizer.save_pretrained(save_dir)

        # Save other related information
        config_path = os.path.join(save_dir, "config.json")
        self.bert.config.to_json_file(config_path)

    def load(self, load_dir, is_eval = True, model_name = "model_bert.pt"):
        # Load tokenizer
        tokenizer = BertTokenizer.from_pretrained(load_dir)

        # Load other related information
        config_path = os.path.join(load_dir, "config.json")
        config = BertConfig.from_json_file(config_path)
        self.bert = BertModel(config)
        if is_eval:
            self.bert.eval()  # Set to evaluation mode

        # Load model weights
        model_path = os.path.join(load_dir, model_name)
        self.load_state_dict(torch.load(model_path))

        # Update the tokenizer
        self.bert.resize_token_embeddings(len(tokenizer))


from tqdm import tqdm
learning_rates = [1e-5] #[5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [8] #[16, 24, 32, 40]

def train(model, train_data, val_data, learning_rate, epochs, batch_size):
    #out_file.write(f"-------------Starting with LR = {learning_rate} and BS = {batch_size}-----------------\n")
    best_acc_val = -99999

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
                #print(train_input)
                train_label = train_label.to(device)

                input_id = train_input['input_ids'].squeeze(1).to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                token_type_id = train_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                print(f'training logits: {logits}')
                print("training outputs")
                print(output)
                optimizer.zero_grad()

                batch_loss = criterion(logits, train_label.long())
                total_loss_train += batch_loss.item()
                print(f'prediction: {torch.argmax(logits, dim=1)}')
                print(f"train labels: {train_label}")
                acc = (torch.argmax(logits, dim=1) == train_label).sum().item()
                total_acc_train += acc


                batch_loss.backward()
                optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)
                token_type_id = val_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                print(val_label)
                print(f'validating logits: {logits}')
                print(f'prediction: {torch.argmax(logits, dim=1)}')

                batch_loss = criterion(logits, val_label.long())
                total_loss_val += batch_loss.item()
                print(output)
                acc = (torch.argmax(logits, dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

        if(total_acc_val / len(val_data) > best_acc_val):
            best_acc_val = total_acc_val / len(val_data)
            model.bert.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))
            tokenizer.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))

            model.save(save_dir = os.path.join(ROOT_PATH, SAVE_MODEL_AT), tokenizer = tokenizer)
            print("Found a better model")
            model.save(save_dir = SAVE_MODEL_AT, tokenizer = tokenizer)

EPOCHS = 8
for LR in learning_rates:
    for bs in batch_sizes:
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = NUM_LABELS)
        model = BertClassifier(load_path = PRETRAINED_PATH)
        train(model, df_train, df_dev, LR, EPOCHS, bs)


Some weights of BertModel were not initialized from the model checkpoint at /content/cellularlint-codes/Pretrained Models/bert_weights/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at /content/cellularlint-codes/Pretrained Models/bert_weights and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized fro

training logits: tensor([[0.1497, 0.4707, 0.3797],
        [0.2063, 0.1981, 0.5956],
        [0.3649, 0.3388, 0.2963],
        [0.0905, 0.3761, 0.5334],
        [0.2920, 0.4067, 0.3013],
        [0.2593, 0.3327, 0.4080],
        [0.3846, 0.3945, 0.2209],
        [0.1342, 0.5164, 0.3494]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.1497, 0.4707, 0.3797],
        [0.2063, 0.1981, 0.5956],
        [0.3649, 0.3388, 0.2963],
        [0.0905, 0.3761, 0.5334],
        [0.2920, 0.4067, 0.3013],
        [0.2593, 0.3327, 0.4080],
        [0.3846, 0.3945, 0.2209],
        [0.1342, 0.5164, 0.3494]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([1, 2, 0, 2, 1, 2, 1, 1], device='cuda:0')
train labels: tensor([0, 1, 2, 0, 2, 1, 2, 2], device='cuda:0')


  2%|▏         | 1/63 [00:01<01:54,  1.85s/it]

training logits: tensor([[0.1645, 0.6726, 0.1629],
        [0.1498, 0.7375, 0.1127],
        [0.2062, 0.3230, 0.4708],
        [0.2764, 0.1937, 0.5299],
        [0.1485, 0.6557, 0.1958],
        [0.4335, 0.2465, 0.3200],
        [0.1616, 0.3109, 0.5275],
        [0.1121, 0.7069, 0.1810]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.1645, 0.6726, 0.1629],
        [0.1498, 0.7375, 0.1127],
        [0.2062, 0.3230, 0.4708],
        [0.2764, 0.1937, 0.5299],
        [0.1485, 0.6557, 0.1958],
        [0.4335, 0.2465, 0.3200],
        [0.1616, 0.3109, 0.5275],
        [0.1121, 0.7069, 0.1810]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([1, 1, 2, 2, 1, 0, 2, 1], device='cuda:0')
train labels: tensor([1, 2, 0, 1, 2, 2, 0, 1], device='cuda:0')


  3%|▎         | 2/63 [00:02<01:09,  1.13s/it]

...

training logits: tensor([[0.9830, 0.0048, 0.0121],
        [0.0114, 0.0089, 0.9797],
        [0.0052, 0.2876, 0.7072],
        [0.0093, 0.0143, 0.9764],
        [0.9770, 0.0017, 0.0212],
        [0.5002, 0.2359, 0.2639],
        [0.0083, 0.0081, 0.9837],
        [0.9791, 0.0036, 0.0173]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.9830, 0.0048, 0.0121],
        [0.0114, 0.0089, 0.9797],
        [0.0052, 0.2876, 0.7072],
        [0.0093, 0.0143, 0.9764],
        [0.9770, 0.0017, 0.0212],
        [0.5002, 0.2359, 0.2639],
        [0.0083, 0.0081, 0.9837],
        [0.9791, 0.0036, 0.0173]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([0, 2, 2, 2, 0, 0, 2, 0], device='cuda:0')
train labels: tensor([0, 2, 1, 2, 0, 1, 2, 0], device='cuda:0')


 98%|█████████▊| 62/63 [00:42<00:00,  1.45it/s]

training logits: tensor([[0.6891, 0.1102, 0.2007],
        [0.0025, 0.0069, 0.9906],
        [0.9652, 0.0182, 0.0166],
        [0.0057, 0.0118, 0.9826]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.6891, 0.1102, 0.2007],
        [0.0025, 0.0069, 0.9906],
        [0.9652, 0.0182, 0.0166],
        [0.0057, 0.0118, 0.9826]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([0, 2, 0, 2], device='cuda:0')
train labels: tensor([2, 1, 0, 2], device='cuda:0')


100%|██████████| 63/63 [00:42<00:00,  1.47it/s]


tensor([2, 1, 0, 2, 0, 1, 1, 0], device='cuda:0')
validating logits: tensor([[0.0069, 0.0083, 0.9849],
        [0.0016, 0.9938, 0.0046],
        [0.9758, 0.0045, 0.0197],
        [0.0063, 0.0056, 0.9881],
        [0.9934, 0.0018, 0.0048],
        [0.0101, 0.9859, 0.0040],
        [0.0081, 0.9878, 0.0041],
        [0.9930, 0.0028, 0.0042]], device='cuda:0')
prediction: tensor([2, 1, 0, 2, 0, 1, 1, 0], device='cuda:0')
tensor([[0.0069, 0.0083, 0.9849],
        [0.0016, 0.9938, 0.0046],
        [0.9758, 0.0045, 0.0197],
        [0.0063, 0.0056, 0.9881],
        [0.9934, 0.0018, 0.0048],
        [0.0101, 0.9859, 0.0040],
        [0.0081, 0.9878, 0.0041],
        [0.9930, 0.0028, 0.0042]], device='cuda:0')
tensor([2, 2, 1, 2, 2, 1, 0, 2], device='cuda:0')
validating logits: tensor([[0.0035, 0.0244, 0.9721],
        [0.0144, 0.0318, 0.9538],
        [0.0081, 0.9868, 0.0051],
        [0.0069, 0.0106, 0.9825],
        [0.0062, 0.0158, 0.9780],
        [0.0053, 0.9898, 0.0049],
        [0.9917,

In [11]:
import pandas as pd
import os
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AdamW, RobertaConfig, RobertaTokenizerFast
#from tokenizers import BertWordPieceTokenizer
from torch import nn
np.random.seed(321)


ROOT_PATH = "/content/cellularlint-codes"
DATA_PATH = os.path.join(ROOT_PATH, "Data/SNLI")
PRETRAINED_PATH = os.path.join(ROOT_PATH, "Pretrained Models/roberta_weights/saved/")
SAVE_MODEL_AT = os.path.join(ROOT_PATH, "saved_models/roberta")
PRETRAINED_TOKENIZER = SAVE_MODEL_AT  # Tokenizer is usually saved in the same model folder
MODEL_PATH = None

df_train = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_train.csv"))
df_dev = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_dev.csv"))
df_test = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_test.csv"))

df_train = df_train.iloc[:500]
df_dev = df_train.copy().iloc[:1000]


#tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_TOKENIZER))
#tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_PATH))
tokenizer = RobertaTokenizerFast.from_pretrained("FacebookAI/roberta-base")


labels = {'contradiction':1,
          'entailment':0,
          'neutral':2,
          }
NUM_LABELS = len(labels)


def roberta_encoding(sequence):
    return tokenizer.encode(sequence, add_special_tokens = False)
def str_to_int_list(data):
    return list(map(int, data))


df_train['token_type'] = df_train['token_type'].str.split()
df_train['token_type'] = df_train['token_type'].apply(str_to_int_list)

df_dev['token_type'] = df_dev['token_type'].str.split()
df_dev['token_type'] = df_dev['token_type'].apply(str_to_int_list)

df_test['token_type'] = df_test['token_type'].str.split()
df_test['token_type'] = df_test['token_type'].apply(str_to_int_list)


df_train['attention_mask'] = df_train['attention_mask'].str.split()
df_train['attention_mask'] = df_train['attention_mask'].apply(str_to_int_list)

df_dev['attention_mask'] = df_dev['attention_mask'].str.split()
df_dev['attention_mask'] = df_dev['attention_mask'].apply(str_to_int_list)

df_test['attention_mask'] = df_test['attention_mask'].str.split()
df_test['attention_mask'] = df_test['attention_mask'].apply(str_to_int_list)


df_train['input_ids'] = df_train['sequence'].apply(roberta_encoding)
df_dev['input_ids'] = df_dev['sequence'].apply(roberta_encoding)
df_test['input_ids'] = df_test['sequence'].apply(roberta_encoding)


max_length = 512
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = []
        self.labels = [labels[label] for label in df['gold_label']]
        for _, row in df.iterrows():
            token_type_ids = row['token_type']
            token_type_ids += [0] * (max_length - len(token_type_ids))
            attention_mask = row['attention_mask']
            attention_mask += [0] * (max_length - len(attention_mask))
            input_ids = tokenizer.encode(
                row['sequence'],
                add_special_tokens=False,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            datadict = {
                'input_ids': input_ids.squeeze(0),
                'token_type_ids': torch.tensor(token_type_ids),
                'attention_mask': torch.tensor(attention_mask)
            }
            self.texts.append(datadict)

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_text = self.texts[idx]
        batch_y = self.labels[idx]
        return batch_text, batch_y


class RobertaClassifier(nn.Module):

    def __init__(self, load_path = None, dropout=0.5):

        super(RobertaClassifier, self).__init__()

        #self.bert = BertModel.from_pretrained(os.path.join(ROOT_PATH, load_path))
        self.roberta = RobertaModel.from_pretrained(load_path)

        #self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, NUM_LABELS)

        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_id, mask, token_type_id):
        _, pooled_output = self.roberta(input_ids= input_id, attention_mask = mask, return_dict = False)

        dropout_output = self.dropout(pooled_output)

        linear_output = self.linear(dropout_output)

        final_layer = self.softmax(linear_output)

        return final_layer

    def save(self, save_dir, tokenizer, model_name = "model_roberta.pt"):

        os.makedirs(save_dir, exist_ok=True)
        # Save model weights
        #model_path = os.path.join(save_dir, model_name)
        #torch.save(self.state_dict(), model_path)
        self.roberta.save_pretrained(save_dir)

        # Save tokenizer
        tokenizer.save_pretrained(save_dir)

        # Save other related information
        #config_path = os.path.join(save_dir, "config.json")
        #self.roberta.config.to_json_file(config_path)

    def load(self, load_dir, is_eval = True, model_name = "model_roberta.pt"):
        # Load tokenizer
        tokenizer = RobertaTokenizer.from_pretrained(load_dir)

        # Load other related information
        config_path = os.path.join(load_dir, "config.json")
        config = RobertaConfig.from_json_file(config_path)
        self.roberta = RobertaModel(config)
        if is_eval:
            self.roberta.eval()  # Set to evaluation mode

        # Load model weights
        model_path = os.path.join(load_dir, model_name)
        self.load_state_dict(torch.load(model_path))

        # Update the tokenizer
        self.roberta.resize_token_embeddings(len(tokenizer))


from tqdm import tqdm
learning_rates = [1e-5] #[5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [8] #[16, 24, 32, 40]

def train(model, train_data, val_data, learning_rate, epochs, batch_size):
    #out_file.write(f"-------------Starting with LR = {learning_rate} and BS = {batch_size}-----------------\n")
    best_acc_val = -99999

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
                #print(train_input)
                train_label = train_label.to(device)

                input_id = train_input['input_ids'].squeeze(1).to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                token_type_id = train_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                #print(f'training logits: {logits}')
                #print("training outputs")
                #print(output)
                optimizer.zero_grad()

                batch_loss = criterion(logits, train_label.long())
                total_loss_train += batch_loss.item()
                #print(f'prediction: {torch.argmax(logits, dim=1)}')
                #print(f"train labels: {train_label}")
                acc = (torch.argmax(logits, dim=1) == train_label).sum().item()
                total_acc_train += acc


                batch_loss.backward()
                optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)
                token_type_id = val_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                #print(val_label)
                #print(f'validating logits: {logits}')
                #print(f'prediction: {torch.argmax(logits, dim=1)}')

                batch_loss = criterion(logits, val_label.long())
                total_loss_val += batch_loss.item()
                #print(output)
                acc = (torch.argmax(logits, dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

        if(total_acc_val / len(val_data) > best_acc_val):
            best_acc_val = total_acc_val / len(val_data)
            #model.bert.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))
            #tokenizer.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))

            model.save(save_dir = SAVE_MODEL_AT, tokenizer = tokenizer)
            print("Found a better model")

EPOCHS = 8
for LR in learning_rates:
    for bs in batch_sizes:
        #model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = NUM_LABELS)
        model = RobertaClassifier(load_path = PRETRAINED_PATH)
        train(model, df_train, df_dev, LR, EPOCHS, bs)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Epochs: 6 | Train Loss:  0.137             | Train Accuracy:  0.380             | Val Loss:  0.136             | Val Accuracy:  0.426
Found a better model


100%|██████████| 63/63 [00:41<00:00,  1.53it/s]


Epochs: 7 | Train Loss:  0.135             | Train Accuracy:  0.406             | Val Loss:  0.130             | Val Accuracy:  0.510
Found a better model


100%|██████████| 63/63 [00:41<00:00,  1.53it/s]


Epochs: 8 | Train Loss:  0.131             | Train Accuracy:  0.490             | Val Loss:  0.119             | Val Accuracy:  0.630
Found a better model


In [12]:
from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification, AdamW, XLNetConfig
#from tokenizers import BertWordPieceTokenizer
from torch import nn
np.random.seed(321)

PRETRAINED_PATH = os.path.join(ROOT_PATH, "Pretrained Models/xlnet_weights/")
SAVE_MODEL_AT = os.path.join(ROOT_PATH, "saved_models/xlnet")
PRETRAINED_TOKENIZER = SAVE_MODEL_AT  # Tokenizer is usually saved in the same model folder
MODEL_PATH = None

df_train = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_train.csv"))
df_dev = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_dev.csv"))
df_test = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_test.csv"))

df_train = df_train.iloc[:500]
df_dev = df_train.copy().iloc[:200]

#tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_TOKENIZER))
#tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_PATH))
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")


labels = {'contradiction':1,
          'entailment':0,
          'neutral':2,
          }
NUM_LABELS = len(labels)


def xlnet_encoding(sequence):
    return tokenizer.encode(sequence, add_special_tokens = False)
def str_to_int_list(data):
    return list(map(int, data))

df_train['token_type'] = df_train['token_type'].str.split()
df_train['token_type'] = df_train['token_type'].apply(str_to_int_list)

df_dev['token_type'] = df_dev['token_type'].str.split()
df_dev['token_type'] = df_dev['token_type'].apply(str_to_int_list)

df_test['token_type'] = df_test['token_type'].str.split()
df_test['token_type'] = df_test['token_type'].apply(str_to_int_list)


df_train['attention_mask'] = df_train['attention_mask'].str.split()
df_train['attention_mask'] = df_train['attention_mask'].apply(str_to_int_list)

df_dev['attention_mask'] = df_dev['attention_mask'].str.split()
df_dev['attention_mask'] = df_dev['attention_mask'].apply(str_to_int_list)

df_test['attention_mask'] = df_test['attention_mask'].str.split()
df_test['attention_mask'] = df_test['attention_mask'].apply(str_to_int_list)

df_train['input_ids'] = df_train['sequence'].apply(xlnet_encoding)
df_dev['input_ids'] = df_dev['sequence'].apply(xlnet_encoding)
df_test['input_ids'] = df_test['sequence'].apply(xlnet_encoding)

max_length = 512
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = []
        self.labels = [labels[label] for label in df['gold_label']]
        for _, row in df.iterrows():
            token_type_ids = row['token_type']
            token_type_ids += [0] * (max_length - len(token_type_ids))
            attention_mask = row['attention_mask']
            attention_mask += [0] * (max_length - len(attention_mask))
            input_ids = tokenizer.encode(
                row['sequence'],
                add_special_tokens=False,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            datadict = {
                'input_ids': input_ids.squeeze(0),
                'token_type_ids': torch.tensor(token_type_ids),
                'attention_mask': torch.tensor(attention_mask)
            }
            self.texts.append(datadict)

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_text = self.texts[idx]
        batch_y = self.labels[idx]
        return batch_text, batch_y

class XLNetClassifier(nn.Module):

    def __init__(self, load_path = None, dropout=0.5):

        super(XLNetClassifier, self).__init__()

        #self.bert = BertModel.from_pretrained(os.path.join(ROOT_PATH, load_path))
        self.xlnet = XLNetModel.from_pretrained(load_path)

        #self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, NUM_LABELS)

        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_id, mask, token_type_id):
        output = self.xlnet(input_ids= input_id, attention_mask = mask, token_type_ids = token_type_id,return_dict = False)
        #_, pooled_output = self.xlnet(input_ids= input_id, attention_mask = mask, return_dict = False)

        pooled_output = output[0][:,-1,:] #Representation from last token
        dropout_output = self.dropout(pooled_output)

        linear_output = self.linear(dropout_output)

        final_layer = self.softmax(linear_output)

        return final_layer

    def save(self, save_dir, tokenizer, model_name = "model_xlnet.pt"):

        os.makedirs(save_dir, exist_ok=True)
        # Save model weights
        #model_path = os.path.join(save_dir, model_name)
        #torch.save(self.state_dict(), model_path)
        self.xlnet.save_pretrained(save_dir)

        # Save tokenizer
        tokenizer.save_pretrained(save_dir)

        # Save other related information
        #config_path = os.path.join(save_dir, "config.json")
        #self.xlnet.config.to_json_file(config_path)

    def load(self, load_dir, is_eval = True, model_name = "model_xlnet.pt"):
        # Load tokenizer
        tokenizer = XLNetTokenizer.from_pretrained(load_dir)

        # Load other related information
        config_path = os.path.join(load_dir, "config.json")
        config = XLNetConfig.from_json_file(config_path)
        self.xlnet = XLNetModel(config)
        if is_eval:
            self.xlnet.eval()  # Set to evaluation mode

        # Load model weights
        model_path = os.path.join(load_dir, model_name)
        self.load_state_dict(torch.load(model_path))

        # Update the tokenizer
        self.xlnet.resize_token_embeddings(len(tokenizer))

from tqdm import tqdm
learning_rates = [1e-5] #[5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [8] #[16, 24, 32, 40]

def train(model, train_data, val_data, learning_rate, epochs, batch_size):
    #out_file.write(f"-------------Starting with LR = {learning_rate} and BS = {batch_size}-----------------\n")
    best_acc_val = -99999

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
                #print(train_input)
                train_label = train_label.to(device)

                input_id = train_input['input_ids'].squeeze(1).to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                token_type_id = train_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                #print(f'training logits: {logits}')
                #print("training outputs")
                #print(output)
                optimizer.zero_grad()

                batch_loss = criterion(logits, train_label.long())
                total_loss_train += batch_loss.item()
                #print(f'prediction: {torch.argmax(logits, dim=1)}')
                #print(f"train labels: {train_label}")
                acc = (torch.argmax(logits, dim=1) == train_label).sum().item()
                total_acc_train += acc


                batch_loss.backward()
                optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)
                token_type_id = val_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                #print(val_label)
                #print(f'validating logits: {logits}')
                #print(f'prediction: {torch.argmax(logits, dim=1)}')

                batch_loss = criterion(logits, val_label.long())
                total_loss_val += batch_loss.item()
                #print(output)
                acc = (torch.argmax(logits, dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

        if(total_acc_val / len(val_data) > best_acc_val):
            best_acc_val = total_acc_val / len(val_data)
            #model.bert.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))
            #tokenizer.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))

            model.save(save_dir = SAVE_MODEL_AT, tokenizer = tokenizer)
            print("Found a better model")

EPOCHS = 8
for LR in learning_rates:
    for bs in batch_sizes:
        #model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = NUM_LABELS)
        model = XLNetClassifier(load_path = PRETRAINED_PATH)
        train(model, df_train, df_dev, LR, EPOCHS, bs)

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 63/63 [01:36<00:00,  1.53s/it]


Epochs: 1 | Train Loss:  0.149             | Train Accuracy:  0.334             | Val Loss:  0.145             | Val Accuracy:  0.315
Found a better model


...

Epochs: 6 | Train Loss:  0.147             | Train Accuracy:  0.332             | Val Loss:  0.142             | Val Accuracy:  0.330


100%|██████████| 63/63 [01:35<00:00,  1.52s/it]


Epochs: 7 | Train Loss:  0.147             | Train Accuracy:  0.310             | Val Loss:  0.143             | Val Accuracy:  0.330


100%|██████████| 63/63 [01:35<00:00,  1.52s/it]


Epochs: 8 | Train Loss:  0.143             | Train Accuracy:  0.364             | Val Loss:  0.143             | Val Accuracy:  0.325


In [13]:
%cd eval
!python3 -W ignore eval.py

/content/cellularlint-codes/eval
Predictions after majority voting: [1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 0, 2, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1]
Metrices successfully calculated. Please check output_metrices.txt file


In [20]:
import os
import re
import glob
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# Set file paths in Google Drive
NET_TYPE = '4G'

input_path = f"/content/cellularlint-codes/Data/Raw_{NET_TYPE}"  # Update with your Drive folder
output_path = f"/content/cellularlint-codes/Data/Processed_{NET_TYPE}"

# Create output directory if not exists
os.makedirs(output_path, exist_ok=True)

class Preprocessor:
    def __init__(self, output_path):
        self.output_path = output_path
        self.output_file = None
        self.line_count = 0
        self.file_count = 0
        self.nas_end_idx = 0

    def find_sections(self, line):
        section_header = re.compile(r'\d(\.\d+)+[A-Za-z]*')
        return section_header.match(line) is not None

    def process_all(self, input_path):
        """
        Processes all .txt files in input_path and saves extracted segments.
        """
        self.file_count = 0
        output_file_path = os.path.join(self.output_path, f'conflict_segments_{NET_TYPE}.txt')

        if os.path.exists(output_file_path):
            with open(output_file_path, 'r', encoding="utf-8") as fp:
                self.line_count = len(fp.readlines())

        self.output_file = open(output_file_path, 'w', encoding="utf-8")
        f_count = 0
        extracted_lines = 0

        for file in glob.glob(os.path.join(input_path, "**/*.txt"), recursive=True):
            print(f"Processing: {os.path.basename(file)}")
            self.process_file(file)

            if f_count == 0:
                self.output_file.write('---------File Ends-------\n')
            f_count += 1

        self.output_file.close()
        print(f"Total extracted lines: {extracted_lines}")

    def process_file(self, file):
        """
        Reads and processes a single file.
        """
        self.file_count += 1
        file_line_count = 0
        file_section_count = -1

        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            text = f.readlines()
            for line in text:
                if self.find_sections(line):
                    self.output_file.write(".\n------\n")
                    file_section_count += 1
                    continue
                if len(line.split()) < 4 or "Editor's Note" in line:
                    continue

                line = self.clean_text(line)

                if line:
                    line_set = line.split('. ')
                    for i, sentence in enumerate(line_set):
                        self.line_count += 1
                        file_line_count += 1
                        if i != len(line_set) - 1:
                            self.output_file.write(sentence + '.\n')
                        else:
                            self.output_file.write(sentence + " ")

            self.output_file.write('\n')
            print(f"Completed {self.file_count} files. {file_section_count} sections added. {file_line_count} lines processed.")

    def clean_text(self, text):
        """
        Cleans a single line of text.
        """
        text = re.sub(r' +', ' ', text)  # Remove multiple spaces
        text = re.sub(r'(\n)+', '', text)  # Remove extra newlines
        text = re.sub(r'^[\.·\-]', '', text)  # Remove leading special characters
        text = re.sub(r'^ ', '', text)  # Remove leading spaces
        text = re.sub(r'[\.:,;\-]*$', '', text)  # Remove trailing special characters
        text = re.sub(r'(\( )', '(', text)  # Fix spacing in parentheses
        text = re.sub(r'( \))', ')', text)
        text = re.sub(r'(\(\))|(\[\])|(\{\})', '', text)  # Remove empty brackets
        text = re.sub(r'(as shown below|[Ss]ee figure below):*\-*', '', text)  # Remove unnecessary text
        text = re.sub(r'([,;:])(\w)', r'\1 \2', text)  # Ensure space after punctuation
        text = re.sub(r'\ue000', '', text, re.UNICODE)  # Remove Unicode artifacts

        return text.strip()

    def pair_up(self, file, num_sentences_per_article=3):
        """
        Segments and pairs up text for TF-IDF similarity analysis.
        """
        corpus = []
        data_string = ""
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            text = f.readlines()
            num_lines = 0
            for line in text:
                if "---------File Ends-------" in line:
                    self.nas_end_idx = len(corpus)
                    continue
                if line in ['.\n', '.', '\n', "------\n"]:
                    if data_string:
                        corpus.append(data_string)
                        num_lines = 0
                    data_string = ""
                else:
                    data_string += line
                    num_lines += 1

                if num_lines == num_sentences_per_article or len(data_string.split()) > 50:
                    corpus.append(data_string)
                    data_string = ""
                    num_lines = 0
            if data_string:
                corpus.append(data_string)

        print(f"Processed {len(corpus)} segments.")

        vect = TfidfVectorizer(min_df=1, stop_words="english")
        tfidf = vect.fit_transform(corpus)
        pairwise_similarity = tfidf * tfidf.T

        return corpus, pairwise_similarity

# ===========================
# Run Preprocessing
# ===========================

dataProcessor = Preprocessor(output_path)
dataProcessor.process_all(input_path)
print(f"NAS End Index: {dataProcessor.nas_end_idx}")

# ===========================
# Pair Up Sentences for Similarity Analysis
# ===========================
conflict_file_path = os.path.join(output_path, f"conflict_segments_{NET_TYPE}.txt")
corpus, pair_sim = dataProcessor.pair_up(conflict_file_path)
print(f"NAS End Index after pairing: {dataProcessor.nas_end_idx}")

# ===========================
# Save Processed Corpus
# ===========================
pickle_file_path = os.path.join(output_path, f"cp_corpus_{NET_TYPE}.pkl")
with open(pickle_file_path, "wb") as fp:
   pickle.dump(corpus, fp)

print(f"Processed corpus saved: {pickle_file_path}")

Processing: Security_4G.txt
Completed 1 files. 120 sections added. 6416 lines processed.
Processing: NAS_4G.txt
Completed 2 files. 890 sections added. 8466 lines processed.
Total extracted lines: 0
NAS End Index: 0
Processed 2596 segments.
NAS End Index after pairing: 720
Processed corpus saved: /content/cellularlint-codes/Data/Processed_4G/cp_corpus_4G.pkl


In [22]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import gensim
import matplotlib.pyplot as plt

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel



NET_TYPE = '4G'
input_path = f"/content/cellularlint-codes/Data/Processed_4G/cp_corpus_4G.pkl"

# Load the corpus
with open(input_path, "rb") as fp:
    corpus_4G = pickle.load(fp)

print(f"Loaded {len(corpus_5G)} sentences from {input_path}")


TYPE = '4G'
SEP_IDX_4G = 720  # Security section boundary
SEP_IDX_5G = 1233  # If evaluating 5G corpus

if TYPE == '4G':
    sentences = corpus_4G
    SEP_IDX = SEP_IDX_4G
elif TYPE == '5G':
    sentences = corpus_5G
    SEP_IDX = SEP_IDX_5G
else:
    raise ValueError("Invalid TYPE. Use '4G' or '5G'.")

# ===========================
# Step: Compute Doc2Vec Embeddings
# ===========================
tagged_data = [TaggedDocument(words=sentence.lower().split(), tags=[str(i)]) for i, sentence in enumerate(sentences)]

# Train Doc2Vec
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Get embeddings & similarity matrix
doc2vec_embeddings = [doc2vec_model.infer_vector(doc.words) for doc in tagged_data]
cosine_sim_matrix_doc2vec = cosine_similarity(doc2vec_embeddings)

print("Doc2Vec similarity matrix computed.")

# ===========================
# Step: Compute SBERT & MPNet Embeddings
# ===========================
def compute_transformer_embeddings(model_name):
    """Computes embeddings using a Sentence Transformer model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenize and embed sentences
    tokenized_sentences = [tokenizer.encode(sentence, padding='max_length', truncation=True, max_length=128) for sentence in sentences]
    input_ids = torch.tensor(tokenized_sentences, dtype=torch.long).to("cuda" if torch.cuda.is_available() else "cpu")

    batch_size = 32
    embeddings = []
    with torch.no_grad():
        for i in range(0, len(input_ids), batch_size):
            batch_inputs = input_ids[i:i+batch_size]
            outputs = model(input_ids=batch_inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0].cpu().numpy()
            embeddings.append(batch_embeddings)

    return np.concatenate(embeddings, axis=0)

# Compute embeddings
sbert_embeddings = compute_transformer_embeddings("sentence-transformers/distilbert-base-nli-stsb-mean-tokens")
cosine_sim_matrix_sbert = cosine_similarity(sbert_embeddings)
print("SBERT similarity matrix computed.")

mpnet_embeddings = compute_transformer_embeddings("sentence-transformers/paraphrase-mpnet-base-v2")
cosine_sim_matrix_use = cosine_similarity(mpnet_embeddings)
print("MPNet similarity matrix computed.")

# ===========================
# Step: Compute Word2Vec Embeddings
# ===========================
tokenized_sentences = [sentence.lower().split() for sentence in sentences]

# Train Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Compute embeddings & similarity
word2vec_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in sentence], axis=0) for sentence in tokenized_sentences])
cosine_sim_matrix_word2vec = cosine_similarity(word2vec_embeddings)

print("Word2Vec similarity matrix computed.")

# ===========================
# Step: Compute TF-IDF Similarity
# ===========================
tfidf_vect = TfidfVectorizer(min_df=1, stop_words="english")
tfidf_matrix = tfidf_vect.fit_transform(sentences)
cosine_sim_matrix_tfidf = (tfidf_matrix * tfidf_matrix.T).toarray()

print("TF-IDF similarity matrix computed.")

# ===========================
# Step: Save Results
# ===========================
output_csv = f"/content/cellularlint-codes/Data/Processed_{NET_TYPE}/tokenizer_comparison_result_{TYPE}.csv"
df = pd.DataFrame({
    "sbert": cosine_sim_matrix_sbert.flatten(),
    "doc2vec": cosine_sim_matrix_doc2vec.flatten(),
    "use": cosine_sim_matrix_use.flatten(),
    "word2vec": cosine_sim_matrix_word2vec.flatten(),
    "tfidf": cosine_sim_matrix_tfidf.flatten()
})
df.to_csv(output_csv, index=False)

print(f"Tokenizer comparison results saved: {output_csv}")



Loaded 2596 sentences from /content/cellularlint-codes/Data/Processed_4G/cp_corpus_4G.pkl
Doc2Vec similarity matrix computed.
SBERT similarity matrix computed.
MPNet similarity matrix computed.
Word2Vec similarity matrix computed.
TF-IDF similarity matrix computed.
Tokenizer comparison results saved: /content/cellularlint-codes/Data/Processed_4G/tokenizer_comparison_result_4G.csv


In [16]:

NET_TYPE = '5G'
local_output_path = f"/content/cellularlint-codes/Data/Processed_{NET_TYPE}/"  # Local output folder
drive_output_path = f"/content/drive/MyDrive/Data/Processed_{NET_TYPE}/"  # Google Drive folder

# Ensure the Drive directory exists
os.makedirs(drive_output_path, exist_ok=True)
print(f"Google Drive save location: {drive_output_path}")

def save_to_drive(filename):
    local_file = os.path.join(local_output_path, filename)
    drive_file = os.path.join(drive_output_path, filename)

    if os.path.exists(local_file):
        os.system(f'cp "{local_file}" "{drive_file}"')
        print(f"Saved: {drive_file}")
    else:
        print(f"ERROR: {filename} not found!")

files_to_save = [
    f"conflict_segments_{NET_TYPE}.txt",
    f"cp_corpus_{NET_TYPE}.pkl",
    f"tokenizer_comparison_result_{NET_TYPE}.csv",
]


for file in files_to_save:
    save_to_drive(file)



Google Drive save location: /content/drive/MyDrive/Data/Processed_5G/
Saved: /content/drive/MyDrive/Data/Processed_5G/conflict_segments_5G.txt
Saved: /content/drive/MyDrive/Data/Processed_5G/cp_corpus_5G.pkl
Saved: /content/drive/MyDrive/Data/Processed_5G/tokenizer_comparison_result_5G.csv


In [24]:
import torch
import os
import pickle
import pandas as pd
import random
from collections import Counter
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast
from transformers import XLNetForSequenceClassification, XLNetTokenizerFast

# ===========================
# STEP 1: Setup and Load Trained Models
# ===========================

# Define model paths
from collections import Counter
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast
from transformers import XLNetForSequenceClassification, XLNetTokenizerFast

# ===========================
# STEP 1: Load the Trained Models
# ===========================

# Define model paths (Modify based on your Drive path)
BERT_PATH = "/content/cellularlint-codes/saved_models/finetune_p1/bert"
ROBERTA_PATH = "/content/cellularlint-codes/saved_models/roberta"
XLNET_PATH = "/content/cellularlint-codes/saved_models/xlnet"

# Load BERT
bert_model = BertForSequenceClassification.from_pretrained(BERT_PATH)
bert_tokenizer = BertTokenizerFast.from_pretrained(BERT_PATH)

# Load RoBERTa
roberta_model = RobertaForSequenceClassification.from_pretrained(ROBERTA_PATH)
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(ROBERTA_PATH)

# Load XLNet
xlnet_model = XLNetForSequenceClassification.from_pretrained(XLNET_PATH)
xlnet_tokenizer = XLNetTokenizerFast.from_pretrained(XLNET_PATH)

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
roberta_model.to(device)
xlnet_model.to(device)

# Set models to evaluation mode
bert_model.eval()
roberta_model.eval()
xlnet_model.eval()

# Load models and tokenizers
def load_model(model_path, model_class, tokenizer_class):
    model = model_class.from_pretrained(model_path).eval()
    tokenizer = tokenizer_class.from_pretrained(model_path)
    return model, tokenizer

print("Loading trained models...")
bert_model, bert_tokenizer = load_model(BERT_PATH, BertForSequenceClassification, BertTokenizerFast)
roberta_model, roberta_tokenizer = load_model(ROBERTA_PATH, RobertaForSequenceClassification, RobertaTokenizerFast)
xlnet_model, xlnet_tokenizer = load_model(XLNET_PATH, XLNetForSequenceClassification, XLNetTokenizerFast)

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
roberta_model.to(device)
xlnet_model.to(device)

# ===========================
# STEP 2: Load `cp_corpus_4G.pkl`
# ===========================

PICKLE_FILE = "/content/cellularlint-codes/Data/Processed_4G/cp_corpus_4G.pkl"

# Load the processed corpus from pickle
with open(PICKLE_FILE, "rb") as f:
    corpus_4G = pickle.load(f)

print(f"Loaded {len(corpus_4G)} sentences from cp_corpus_4G.pkl")

# ===========================
# STEP 3: Batch Prediction Function
# ===========================

def batch_predict(texts, model, tokenizer, batch_size=8):
    """
    Predicts batch labels using a trained model.
    Prevents OOM by using a smaller batch size.
    """
    predictions = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1).tolist()

        predictions.extend(preds)
        torch.cuda.empty_cache()  #Clear unused GPU memory

    return predictions

# Define class labels
LABEL_MAP = {0: "Consistent", 1: "Inconsistent", 2: "Neutral"}  # Adjust if needed

# ===========================
# STEP 4: Perform Predictions and Apply Majority Voting
# ===========================

print("\nRunning Majority Voting on cp_corpus_4G.pkl...")

#Reduce batch size to avoid OOM
# Get predictions in batch mode
bert_preds = batch_predict(corpus_4G, bert_model, bert_tokenizer)
roberta_preds = batch_predict(corpus_4G, roberta_model, roberta_tokenizer)
xlnet_preds = batch_predict(corpus_4G, xlnet_model, xlnet_tokenizer)

# Convert numeric predictions to labels
bert_preds = [LABEL_MAP[p] for p in bert_preds]
roberta_preds = [LABEL_MAP[p] for p in roberta_preds]
xlnet_preds = [LABEL_MAP[p] for p in xlnet_preds]

# Majority voting function (Fix for randomness)
def majority_vote(list1, list2, list3):
    majority_list = []
    for a, b, c in zip(list1, list2, list3):
        vote_count = Counter([a, b, c])
        if vote_count.most_common(1)[0][1] == 1:  # If all predictions are different, choose randomly
            majority_element = random.choice([a, b, c])
        else:
            majority_element = vote_count.most_common(1)[0][0]  # Most frequent label
        majority_list.append(majority_element)
    return majority_list

# Perform majority voting
final_predictions = majority_vote(bert_preds, roberta_preds, xlnet_preds)

# Store results in a DataFrame
df_results = pd.DataFrame({
    "Text": corpus_5G,
    "BERT Prediction": bert_preds,
    "RoBERTa Prediction": roberta_preds,
    "XLNet Prediction": xlnet_preds,
    "Final Prediction": final_predictions
})

print("Majority voting completed!")
# ===========================
# STEP 5: Save Results to CSV
# ===========================

SAVE_PATH = "/content/drive/MyDrive/majority_voting_cp_corpus_4G.csv"
df_results.to_csv(SAVE_PATH, index=False)

print(f"Majority voting results saved to: {SAVE_PATH}")

# ===========================
# STEP 6: Display Sample Predictions
# ===========================
print("\nSample Predictions:")
print(df_results.head(10))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/finetune_p1/bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/xlnet and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summa

Loading trained models...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/finetune_p1/bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at /content/cellularlint-codes/saved_models/xlnet and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summa

Loaded 2596 sentences from cp_corpus_4G.pkl

Running Majority Voting on cp_corpus_4G.pkl...
Majority voting completed!
Majority voting results saved to: /content/drive/MyDrive/majority_voting_cp_corpus_4G.csv

Sample Predictions:
                                                Text BERT Prediction  \
0  4 Overview of Security Architecture Five secur...    Inconsistent   
1  They combine the functionalities of ME and AN ...      Consistent   
2  The statements relating to eNBs in clause 5.1 ...      Consistent   
3  User identity confidentiality is as defined by...      Consistent   
4  When NAS confidentiality protection is beyond ...      Consistent   
5  Entity authentication is as defined by TS 33.1...      Consistent   
6  Ciphering may be provided to RRC-signalling to...    Inconsistent   
7  NAS signalling confidentiality is an operator ...      Consistent   
8  This shall be accomplished by the network by s...    Inconsistent   
9  All algorithms specified in this clause are al.